In [ ]:
# test commit

In [1]:
from data_sampler import WeightedSampler
from dataset import CARLADataset, CARLADatasetMultiProcessing
from data_preprocessing import preprocessing
from torch.utils.data import DataLoader
from torchvision import transforms
import numpy as np
import torch
import os
import pandas as pd
import matplotlib.pyplot as plt

import cv2

/Users/julianvonklitzing/miniforge3/envs/carla/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
path_data = os.path.join("..", "data", "Dataset Ege")
# path_data = os.path.join("..", "data", "data")

config = {"used_inputs": ["rgb", "measurements"], 
        "used_measurements": ["speed", "steer", "throttle"],
        "seq_len": 1
        }

dataset = CARLADataset(root_dir=path_data, config=config)
# dataset_mp = CARLADatasetMultiProcessing(root_dir=path_data, config=config)
weighted_sampler = WeightedSampler(dataset=dataset)
print(dataset.__len__())

Varying number files among input types: ../data/Dataset Ege/Town10HD_Scenario10_route3_11_28_17_28_17/topdown
Varying number files among input types: ../data/Dataset Ege/Town10HD_Scenario10_route3_11_28_17_28_17/label_raw
Varying number files among input types: ../data/Dataset Ege/Town10HD_Scenario10_route3_11_28_17_28_17/depth
217


In [7]:
dl = DataLoader(dataset=dataset, batch_size=16, num_workers=0, sampler=weighted_sampler)
# dl_mp = DataLoader(dataset=dataset_mp, batch_size=16, num_workers=3, sampler=weighted_sampler)

In [8]:
dataset.data_shapes

[[3, 160, 960], None]

In [9]:
b = next(iter(dl))

In [4]:
count = 0
for batch in dl:
    # preprocessing
    for key in preprocessing:
        batch[key] = preprocessing[key](batch[key])
    count += 1
print(count)

14


In [5]:
# Derive statistics for our dataset that can be used in Normalize() method to get approx. zero mean & var 1. 
mean = 0
std = 0
count = 0
for batch in dl:
    mean = (count * mean + batch["rgb"].shape[0] * batch["rgb"].mean(axis=[0, 1, 3, 4])) / (count + batch["rgb"].shape[0])
    std = (count * std + batch["rgb"].shape[0] * batch["rgb"].std(axis=[0, 1, 3, 4])) / (count + batch["rgb"].shape[0])
    count += batch["rgb"].shape[0]
print(count)

217


In [15]:
dataset.__getitem__(1)

ValueError: could not broadcast input array from shape (27352,4) into shape (27498,4)